In [13]:
include(joinpath(dirname(Base.active_project()), "ClassImport.jl"))
using HDF5, JSON
using .ClassImport: GreedyRegression, Integrate, ReportModel, PickSubdomains, EnvelopePol, FUNCTION_PATH, HDF5_PATH, JSON_PATH

p::Array{Float64} = Float64[]
t::Array{Float64} = Float64[]
u::Array{Float64} = Float64[]
v::Array{Float64} = Float64[]
x::Array{Float64} = Float64[]
y::Array{Float64} = Float64[]
Bx::Array{Float64} = Float64[]
By::Array{Float64} = Float64[]

h5open(joinpath(HDF5_PATH, "mhd_rk4_naive_result.hdf5"), "r") do file
    global p = read(file, "result")["value"]["p"]["value"]
    global t = read(file, "result")["value"]["t"]["value"]
    global u = read(file, "result")["value"]["u"]["value"]
    global v = read(file, "result")["value"]["v"]["value"]
    global x = read(file, "result")["value"]["x"]["value"]
    global y = read(file, "result")["value"]["y"]["value"]
    global Bx = read(file, "result")["value"]["Bx"]["value"]
    global By = read(file, "result")["value"]["By"]["value"]
end

128×128×100 Array{Float64, 3}:
[:, :, 1] =
 -2.93305e-16  0.0490677  0.0980171  …  -0.14673   -0.0980171  -0.0490677
 -6.94566e-16  0.0490086  0.0978991     -0.146554  -0.0978991  -0.0490086
 -1.16322e-15  0.0488314  0.0975452     -0.146024  -0.0975452  -0.0488314
 -9.97714e-16  0.0485366  0.0969563     -0.145142  -0.0969563  -0.0485366
 -4.57079e-16  0.0481249  0.0961338     -0.143911  -0.0961338  -0.0481249
 -3.3472e-16   0.0475972  0.0950797  …  -0.142333  -0.0950797  -0.0475972
 -6.53865e-16  0.0469548  0.0937966     -0.140412  -0.0937966  -0.0469548
 -4.5753e-16   0.0461994  0.0922875     -0.138153  -0.0922875  -0.0461994
  1.61526e-16  0.0453326  0.090556      -0.135561  -0.090556   -0.0453326
 -5.49253e-17  0.0443567  0.0886064     -0.132643  -0.0886064  -0.0443567
 -1.05745e-15  0.0432738  0.0864434  …  -0.129405  -0.0864434  -0.0432738
 -1.16594e-15  0.0420867  0.0840721     -0.125855  -0.0840721  -0.0420867
 -1.35469e-16  0.0407983  0.0814983     -0.122002  -0.0814983  -0.040

In [14]:
nl::Int = 2 # under-estimate this
nw::Int = 128 # number of domains we integrate over 
dof::Int = 1 # scalars have one degree of freedom
dim::Int = 3 # how many dimensions does our data have?
env_pow::Int = 4 # weight is (1-x^2)^power
size_vec::Vector{Int} = [64,64,64] # how many gridpoints should we use per integration?
buffer::Int = 0 # Don't use points this close to boundary

pol::Matrix{Float64} = EnvelopePol.envelope_pol(env_pow, dim)

size_of_data::Vector{Int} = [size(u,1), size(u,2), size(u,3)]
seed::Int = 1
corners::Matrix{Int} = PickSubdomains.pick_subdomains(size_of_data, size_vec, buffer, nw, seed)

grid::Vector{Vector{Float64}} = [vec(y), vec(x), vec(t)]
size_of_data

3-element Vector{Int64}:
 128
 128
 100

In [16]:
using Statistics

G::Matrix{Float64} = zeros(Float64, dof*nw, 0)
labels::Vector{String} = String[]
scales::Vector{Float64} = Float64[]

function add_library_term(label::AbstractString, data::AbstractArray, derivs::AbstractVector{Int64}, scale::AbstractFloat)
    col::Vector{Float64} = Integrate.poly_integrate(data, derivs, grid, corners, size_vec, pol)

    global G = hcat(G, col)
    push!(labels, label)
    push!(scales, scale)
end

# flow terms
add_library_term("∂u/∂x", u, [2], 1.)
add_library_term("∂v/∂x", v, [2], 1.)
add_library_term("∂u/∂y", u, [1], 1.)
add_library_term("∂v/∂y", v, [1], 1.)
add_library_term("∂u/∂t", u, [3], 1.)
add_library_term("∂v/∂t", v, [3], 1.)
add_library_term("∂u^2/∂x", u .* u, [2], 1.)
add_library_term("∂uv/∂x", u .* v, [2], 1.)
add_library_term("∂uv/∂y", u .* v, [1], 1.)
add_library_term("∂v^2/∂y", v .* v, [1], 1.)

# flow terms second derivatives
add_library_term("∂^2u/∂x^2", u, [2,2], 1.)
add_library_term("∂^2u/∂y^2", u, [1,1], 1.)
add_library_term("∂^2v/∂x^2", v, [2,2], 1.)
add_library_term("∂^2v/∂y^2", v, [1,1], 1.)

# pressure terms
add_library_term("∂P/∂x", p, [2], 1.)
add_library_term("∂P/∂y", p, [1], 1.)

# B field terms
add_library_term("∂Bx^2/∂x", Bx .* Bx, [2], 1.)
add_library_term("∂Bx^2/∂y", Bx .* Bx, [1], 1.)
add_library_term("∂By^2/∂x", By .* By, [2], 1.)
add_library_term("∂By^2/∂y", By .* By, [1], 1.)
add_library_term("∂BxBy/∂x", Bx .* By, [2], 1.)
add_library_term("∂BxBy/∂y", Bx .* By, [1], 1.)
add_library_term("∂Bx/∂t", Bx, [3], 1.)
add_library_term("∂By/∂t", By, [3], 1.)

# B field second derivatives
add_library_term("∂^2Bx/∂x^2", Bx, [2, 2], 1.)
add_library_term("∂^2By/∂x^2", By, [2, 2], 1.)
add_library_term("∂^2Bx/∂y^2", Bx, [1, 1], 1.)
add_library_term("∂^2By/∂y^2", By, [1, 1], 1.)

# weird terms
add_library_term("∂uBy/∂x", u .* By, [2], 1.)
add_library_term("∂vBx/∂x", v .* Bx, [2], 1.)
add_library_term("∂uBy/∂y", u .* By, [1], 1.)
add_library_term("∂vBx/∂y", v .* Bx, [1], 1.)

norm_vec::Vector{Float64} = Integrate.poly_integrate(0*u .+ 1, Int64[], grid, corners, size_vec, pol)
G ./= norm_vec
G ./= scales'

G

128×32 Matrix{Float64}:
 -0.00286905   0.140737    -0.134245    …  -0.0147259     0.00519588
 -0.0143862   -0.112621     0.0686435      -0.0777757     0.0216863
 -0.112567    -0.107111     0.119581        0.075781     -0.0578278
 -0.0538018    0.0774715   -0.0993283       0.0753598    -0.0242737
 -0.0336048   -0.111136     0.131696        0.0388557    -0.0335506
 -0.0764833   -0.0172035   -0.015475    …  -0.00909417    0.0654145
 -0.0270416    0.020113     0.0163525      -0.00395647   -0.0103147
 -0.136537    -0.00425859   0.0043003       0.0270975    -0.112786
  0.0149863   -0.022671     0.056659       -0.0365774     0.00558626
 -0.0363514    0.195931    -0.178006       -0.0828006     0.0290356
 -0.0865418    0.00568599  -0.00195999  …   0.000930942  -0.0130246
  0.06242      0.0181474   -0.021199        0.0179183    -0.00965672
 -0.108761     0.0042535    0.00211952     -0.00464736    0.0739615
  ⋮                                     ⋱   ⋮            
 -0.0143846   -0.176087     0.14

In [10]:
using LinearAlgebra

gamma::Float64 = 1.5

for i::Int = 1:5
    cs::Matrix{Float64}, residuals::Vector{Float64}, _ = GreedyRegression.greedy_regression(G)
    k::Int = ReportModel.report_identified_model(cs, residuals, scales, labels, gamma)

    col_norms::Vector{Float64} = [norm(G[:, j] .* cs[j, k]) for j = 1:size(G, 2)]
    kill::Int = argmax(col_norms)
    G = G[:, setdiff(1:end, kill)]
    labels = labels[setdiff(1:end, kill)]
    scales = scales[setdiff(1:end, kill)]
end

∂u/∂x + ∂v/∂y = 0
∂v/∂t + ∂uv/∂x + ∂v^2/∂y - 0.011∂^2v/∂x^2 - 0.013∂^2v/∂y^2 + ∂P/∂y - ∂By^2/∂y - ∂BxBy/∂x = 0
∂Bx/∂t - 0.019∂^2Bx/∂x^2 - 0.021∂^2Bx/∂y^2 - ∂uBy/∂y + ∂vBx/∂y = 0
∂u/∂t + ∂u^2/∂x + ∂uv/∂y - 0.013∂^2u/∂x^2 - 0.01∂^2u/∂y^2 + 0.99∂P/∂x - 0.99∂Bx^2/∂x - 0.99∂BxBy/∂y = 0
∂By/∂t - 0.021∂^2By/∂x^2 - 0.019∂^2By/∂y^2 + ∂uBy/∂x - ∂vBx/∂x = 0
